### AutoGPTQ
[AutoGPTQ](https://github.com/PanQiWei/AutoGPTQ) 是個能夠簡單進行 GPTQ 量化的套件，此套件已被整合在 Transformers 裡面。

#### 1. 若要使用 GPTQ 量化，需要先安裝相關套件：

In [ ]:
%%bash
pip install optimum auto-gptq

#### 2. 透過 `GPTQConfig` 就可以對模型進行量化：
* 參數 `bits` 為量化的位元大小，可以是 2、3、4 或 8 位元。
* 因為 GPTQ 是逐層量化的演算法，所以可以設定 `device_map="cpu"`，在量化的過程中把模型權重都先放在 CPU 記憶體裡面，只有在某一層需要量化時，再拿出來放進 GPU 記憶體裡面做量化，完成量化後再放回 CPU 記憶體。
    * 如此一來，再大的模型都不怕了，只要確保其中一層權重能放的下，就能對整個模型進行量化囉!
* 使用 RTX 3090 將 Llama 3 8B 量化至 4-Bit， 大約只需要花費 7 分鐘左右。
* GPTQ 格式的模型能直接存在硬碟裡面，原本 Llana 3 8B 佔用了 15 GB 的硬碟空間，進行 GPTQ 量化後縮小到了 5.4 GB 而已!
* 注意：
    * `model_seqlen` 會影響 GPU 記憶體的使用量，如果會發生 CUDA OOM 的話可以試著調低這個數字。
    * 如果會遇到 `torch._C._LinAlgError` 錯誤的話，可以嘗試提高 `damp_percent` 的值。
    * 若是會遇到 `RuntimeError: shape '...' is invalid for input of size...` 的錯誤，則可以將 `desc_act` 設為 True 避免此問題。

In [ ]:
from transformers import GPTQConfig
from transformers import LlamaForCausalLM as ModelCls
from transformers import LlamaTokenizerFast as TkCls

model_id = "meta-llama/Meta-Llama-3-8B"

tk: TkCls = TkCls.from_pretrained(model_id)
gptg_config = GPTQConfig(
    bits=4,             # 可以是2, 3, 4, 8
    tokenizer=tk,
    dataset="wikitext2",
    desc_act=False,
    # 提高 damp_percent 可以避免 torch._C._LinAlgError
    damp_percent=0.1,
    model_seqlen=2048,  # 影響 GPU 使用量
)

model: ModelCls = ModelCls.from_pretrained(
    model_id,
    device_map="cpu",   # 設定 CPU Offload
    low_cpu_mem_usage=True,
    quantization_config=gptg_config,
)

output_dir = "Models/Llama3-8B-Inst-GPTQ"
model.save_pretrained(output_dir)
tk.save_pretrained(output_dir)

#### 3. 也可以使用自己的文本進行 GPTQ 量化
* 在 GPTQ 原論文裡面，是從預訓練資料裡面，隨機挑選 128 筆固定長度的文本做校準。
* 注意：
    * 雖然 GPTQ 可以透過校準資料集提昇量化的精準度，但也因此產生了過度擬合於校準資料集的問題。所以慎選校準資料集，以及量化後的完整評估相當重要。

In [ ]:
dataset = ["Hello, GPTQ!", "Goodbye, GPTQ!"]
gptq_configm= GPTQConfig(bits=4, tokenizer=tk, dataset=dataset)

#### 4. 讀取已經做過 GPTQ 量化的模型時，不用再給一次 GPTQConfig，直接讀取即可：

In [ ]:
from transformers import LlamaForCausalLM as ModelCls

model: ModelCls = ModelCls.from_pretrained(
    "Models/Liama3-8B-Inst-GPTO",
    device_map="auto",
)

可以選擇直接去 HF Hub 上面搜尋 GPTQ，就可以找到許多別人已經做完量化的權重。
* 參考量化大神 TheBloke 的 [Script](https://github.com/PanQiWei/AutoGPTQ/issues/179#issuecomment-1611257490)
* HF Hub 上面已經做完 GPTQ 訓練，例如 TheBloke 大神就上傳相當多[使用 GPTQ 量化過的模型](https://huggingface.co/models?search=thebloke%20gptq)。